In [1]:
%load_ext autoreload
%autoreload 2
from commons import *

In [2]:
# Guitarlele tuning
# guitar = Guitar(tuning=['A2','D3','G3','C3','E4','A5'])
guitar = Guitar()
guitar.printFretboardDiagram()

 E  █ F   | F#  | G   | G#  | A   | A#  | B   | C   | C#  | D   | D#  | E   | F   | F#  | G   | G#  | A   | A#  | B   |
 B  █ C   | C#  | D   | D#  | E   | F   | F#  | G   | G#  | A   | A#  | B   | C   | C#  | D   | D#  | E   | F   | F#  |
 G  █ G#  | A   | A#  | B   | C   | C#  | D   | D#  | E   | F   | F#  | G   | G#  | A   | A#  | B   | C   | C#  | D   |
 D  █ D#  | E   | F   | F#  | G   | G#  | A   | A#  | B   | C   | C#  | D   | D#  | E   | F   | F#  | G   | G#  | A   |
 A  █ A#  | B   | C   | C#  | D   | D#  | E   | F   | F#  | G   | G#  | A   | A#  | B   | C   | C#  | D   | D#  | E   |
 E  █ F   | F#  | G   | G#  | A   | A#  | B   | C   | C#  | D   | D#  | E   | F   | F#  | G   | G#  | A   | A#  | B   |
                              •           •           •                 ••                •           •           •   


In [73]:
natural_scale   = [0, 2, 4, 5, 7, 9, 11]
# minor_melodic   = [0, 2, 3, 5, 7, 9, 10]
# minor_harmonic  = [0, 2, 3, 5, 7, 8, 11]
# major_harmonic  = [0, 2, 3, 5, 7, 8, 11]

def parseScaleListWH(notesList):
    res = [0]
    for i in notesList:
        halfs = i.count('H')
        fulls = i.count('W')
        res.append(res[-1] + halfs + fulls*2)
    return res

def parseScaleList(notesList, reference):
    if 'W' in notesList[0] or 'H' in notesList[0]:
        return parseScaleListWH(notesList)

    res = []
    for i in notesList:
        i = i.replace(' ','')
        flats = i.count('b')
        sharps = i.count('#')
        num = int(i.replace('b','').replace('#','')) - 1
        res.append(reference[num] - flats + sharps)
    res.sort()
    return res

class Scale:
    def __init__(self, progression, reference = natural_scale): #reference is major
        if isinstance(progression, list):
            if isinstance(progression[0], int):
                self.progression = progression
            else:
                self.progression = parseScaleList(progression, reference)
        elif isinstance(progression, str):
            self.progression = parseScaleList(progression.split(','), reference)

    def __str__(self):
        return str(self.progression)

    def __getitem__(self, item):
        return self.progression[item]
    
    def __len__(self):
        return len(self.progression)

    def mode(self, num=1):
        assert num > 0 and num <= len(self.progression)
        num -= 1
        root = self.progression[num]
        
        return Scale([i - root for i in self.progression[num:]] \
            + [i - root + 12 for i in self.progression[:num]])


print(Scale('1,2#,3,4,5,6'))
print(Scale('WH,H,H,W,W'))

[0, 3, 4, 5, 7, 9]
[0, 3, 4, 5, 7, 9]


In [69]:
scales={}
scales['major'] = Scale([0, 2, 4, 5, 7, 9, 11])
scales['dorian'] = scales['major'].mode(2)
scales['phrygian'] = scales['major'].mode(3)
scales['lydian'] = scales['major'].mode(4)
scales['mixolydian'] = scales['major'].mode(5)
# scales['aeolian'] = \
scales['natural minor'] = scales['major'].mode(6)
scales['locrian'] = scales['major'].mode(7)


print(NoteSet(scales['major'], Note('C3')))
print(NoteSet(scales['dorian'], Note('D3')))
print(NoteSet(scales['phrygian'], Note('E3')))
print(NoteSet(scales['lydian'], Note('F3')))
print(NoteSet(scales['mixolydian'], Note('G3')))
print(NoteSet(scales['natural minor'], Note('A4')))
print(NoteSet(scales['locrian'], Note('B4')))


C D E F G A B
D E F G A B C
E F G A B C D
F G A B C D E
G A B C D E F
A B C D E F G
B C D E F G A


In [70]:
scales['melodic minor'] = [0, 2, 3, 5, 7, 9, 10]
scales['dorian b2'] = mode(scales['melodic minor'], 2)
scales['lydian augmented'] = mode(scales['melodic minor'], 3)
scales['lydian dominant'] = mode(scales['melodic minor'], 4)
scales['aeolian dominant'] = mode(scales['melodic minor'], 5)
scales['half diminished'] = mode(scales['melodic minor'], 6)
scales['altered'] = mode(scales['melodic minor'], 7)


print(NoteSet(scales['melodic minor'], Note('C3')))
print(NoteSet(scales['dorian b2'], Note('D3')))
print(NoteSet(scales['lydian augmented'], Note('D#3')))
print(NoteSet(scales['lydian dominant'], Note('F3')))
print(NoteSet(scales['aeolian dominant'], Note('G3')))
print(NoteSet(scales['half diminished'], Note('A4')))
print(NoteSet(scales['altered'], Note('A#4')))

C D D# F G A A#
D D# F G A A# C
D# F G A A# C D
F G A A# C D D#
G A A# C D D# F
A A# C D D# F G
A# C D D# F G A


In [71]:
scales['harmonic minor'] = [0, 2, 3, 5, 7, 8, 11]
scales['locrian nat6'] = mode(scales['harmonic minor'], 2)
scales['major #5'] = mode(scales['harmonic minor'], 3)
scales['dorian #4'] = mode(scales['harmonic minor'], 4)
scales['phrygian dominant'] = mode(scales['harmonic minor'], 5)
scales['lydian #2'] = mode(scales['harmonic minor'], 6)
scales['altered dominant bb7'] = mode(scales['harmonic minor'], 7)


print(NoteSet(scales['harmonic minor'], Note('C3')))
print(NoteSet(scales['locrian nat6'], Note('D3')))
print(NoteSet(scales['major #5'], Note('D#3')))
print(NoteSet(scales['dorian #4'], Note('F3')))
print(NoteSet(scales['phrygian dominant'], Note('G3')))
print(NoteSet(scales['lydian #2'], Note('G#3')))
print(NoteSet(scales['altered dominant bb7'], Note('B4')))

C D D# F G G# B
D D# F G G# B C
D# F G G# B C D
F G G# B C D D#
G G# B C D D# F
G# B C D D# F G
B C D D# F G G#


In [72]:
scales['harmonic major'] = [0, 2, 4, 5, 7, 8, 11]
scales['dorian b5'] = mode(scales['harmonic major'], 2)
scales['phrygian b4'] = mode(scales['harmonic major'], 3)
scales['lydian b4'] = mode(scales['harmonic major'], 4)
scales['mixolydian b2'] = mode(scales['harmonic major'], 5)
scales['lydian augmented #2'] = mode(scales['harmonic major'], 6)
scales['locrian bb7'] = mode(scales['harmonic major'], 7)


print(NoteSet(scales['harmonic major'], Note('C3')))
print(NoteSet(scales['dorian b5'], Note('D3')))
print(NoteSet(scales['phrygian b4'], Note('E3')))
print(NoteSet(scales['lydian b4'], Note('F3')))
print(NoteSet(scales['mixolydian b2'], Note('G3')))
print(NoteSet(scales['lydian augmented #2'], Note('G#3')))
print(NoteSet(scales['locrian bb7'], Note('B4')))

C D E F G G# B
D E F G G# B C
E F G G# B C D
F G G# B C D E
G G# B C D E F
G# B C D E F G
B C D E F G G#


In [57]:
scales['diminished'] = [0, 2, 3, 5, 6, 8, 9, 11]
scales['inverse diminished'] = mode(scales['diminished'], 2)

print(NoteSet(scales['diminished'], Note('C3')))
print(NoteSet(scales['inverse diminished'], Note('D3')))
print(NoteSet(scales['diminished'], Note('D#3')))
print(NoteSet(scales['inverse diminished'], Note('F3')))
print(NoteSet(scales['diminished'], Note('F#3')))
print(NoteSet(scales['inverse diminished'], Note('G#3')))
print(NoteSet(scales['diminished'], Note('A4')))
print(NoteSet(scales['inverse diminished'], Note('B4')))

C D D# F F# G# A B
D D# F F# G# A B C
D# F F# G# A B C D
F F# G# A B C D D#
F# G# A B C D D# F
G# A B C D D# F F#
A B C D D# F F# G#
B C D D# F F# G# A


In [56]:
scales['augmented'] = [0, 3, 4, 7, 8, 11]
scales['inverse augmented'] = mode(scales['augmented'], 2)

print(NoteSet(scales['augmented'], Note('C3')))
print(NoteSet(scales['inverse augmented'], Note('D#3')))
print(NoteSet(scales['augmented'], Note('E3')))
print(NoteSet(scales['inverse augmented'], Note('G3')))
print(NoteSet(scales['augmented'], Note('G#3')))
print(NoteSet(scales['inverse augmented'], Note('B4')))

C D# E G G# B
D# E G G# B C
E G G# B C D#
G G# B C D# E
G# B C D# E G
B C D# E G G#
